In [0]:
dbutils.fs.put("FileStore/tables/sequence.csv",""" EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE
7369,SMITH,CLERK,7902,17-12-1980,800,null,20,2022-01-01
7499,ALLEN,SALESMAN,7698,20-02-1981,1600,300,30,2022-01-01
7521,WARD,SALESMAN,7698,22-02-1981,1250,500,30,2022-01-01
7566,JONES,MANAGER,7839,04-02-1981,2975,null,20,2022-01-05
7654,MARTIN,SALESMAN,7698,21-09-1981,1250,1400,30,2022-01-03
7698,SGR,MANAGER,7839,05-01-1981,2850,null,30,2022-01-04
7782,RAVI,MANAGER,7839,06-09-1981,2450,null,10,2022-01-02
7788,SCOTT,ANALYST,7566,19-04-1987,3000,null,20,2022-01-02
7839,KING,PRESIDENT,null,01-11-1981,5000,null,10,2022-01-02
7844,TURNER,SALESMAN,7698,09-08-1981,1500,0,30,2022-01-02
7876,ADAMS,CLERK,7788,23-05-1987,1100,null,20,2022-01-03
7900,JAMES,CLERK,7698,12-03-1981,950,null,30,2022-01-03
7902,FORD,ANALYST,7566,12-03-1981,3000,null,20,2022-01-03
7934,MILLER,CLERK,7782,01-03-1982,1300,null,10,2022-01-03
1234,SEKHAR,doctor,7777,null,667,78,80,2022-01-03
7369,SMITH,CLERK,7902,17-12-1980,800,null,20,2022-01-04
7499,ALLEN,SALESMAN,7698,20-02-1981,1600,300,30,2022-01-04
7521,WARD,SALESMAN,7698,22-02-1981,1250,500,30,2022-01-04
7566,JONES,MANAGER,7839,04-02-1981,2975,null,20,2022-01-04
7654,MARTIN,SALESMAN,7698,21-09-1981,1250,1400,30,2022-01-05
7698,SGR,MANAGER,7839,05-01-1981,2850,null,30,2022-01-05
7782,RAVI,MANAGER,7839,06-09-1981,2450,null,10,2022-01-05
7788,SCOTT,ANALYST,7566,19-04-1987,3000,null,20,2022-01-06
7839,KING,PRESIDENT,null,01-11-1981,5000,null,10,2022-01-06
7844,TURNER,SALESMAN,7698,09-08-1981,1500,0,30,2022-01-06
7876,ADAMS,CLERK,7788,23-05-1987,1100,null,20,2022-01-06
7900,JAMES,CLERK,7698,12-03-1981,950,null,30,2022-01-07
7902,FORD,ANALYST,7566,12-03-1981,3000,null,20,2022-01-07
7934,MILLER,CLERK,7782,01-03-1982,1300,null,10,2022-01-07
1234,RAM,CLERK,7457,null,494,588,80,2022-01-07
null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null""",True)

Wrote 1918 bytes.
Out[37]: True

In [0]:
# crc37 will not generate seq if there are null values
df=spark.read.option("nullValue","null").csv("/FileStore/tables/sequence.csv",header=True,inferSchema=True).dropna(how='all')
df.columns
#display(df)


Out[52]: [' EMPNO',
 'ENAME',
 'JOB',
 'MGR',
 'HIREDATE',
 'SAL',
 'COMM',
 'DEPTNO',
 'UPDATED_DATE']

In [0]:
#strip will remove trailing spaces in columns 
df3=df.select([col(column).alias(column.strip()) for column in df.columns])
df3.columns
#display(df1)

Out[53]: ['EMPNO',
 'ENAME',
 'JOB',
 'MGR',
 'HIREDATE',
 'SAL',
 'COMM',
 'DEPTNO',
 'UPDATED_DATE']

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
# monotonically_increasing_id,Row_Number--only for small data sets,crc32 upto million records then generate duplicates,MD5 will work upto 10 million records,SHA1,SHA2 --in prod
df2=df3.withColumn("Seq_mono",monotonically_increasing_id()+1)\
      .withColumn("seq_row",row_number().over(Window.partitionBy(lit('')).orderBy(lit(''))))\
      .withColumn("CRC32_KEY",crc32(col("EMPNO").cast("string")))\
      .withColumn("MD5",md5(col("EMPNO").cast("string")))\
      .withColumn("SHA2",sha2(col("EMPNO").cast("string"),256))
display(df2)

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE,Seq_mono,seq_row,CRC32_KEY,MD5,SHA2
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-01,1,1,3163315632,0d7f9017fbda691900187b22404b8a1f,c59f438f16c5a409eb2a040b299e82de37503321b9cbfec4fb351547261dd1b1
7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,2022-01-01,2,2,1046173690,7a2b33c672ce223b2aa5789171ddde2f,4427dc2e32a1d099dbe2e3c093a8726e0ea72b9422c36a99915dae1d31e9385f
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-01,3,3,3535170612,e1e1f667ce4596e5644be6fab627c226,74ed8ca63e8b4fb8b8ac06e8df400f098b0b09cf1b89c8a331e72e1919b57bd4
7566,JONES,MANAGER,7839,1981-02-04,2975,null,20,2022-01-05,4,4,683555987,b937384a573b94c4d7cc6004c496f919,b2ca4f93866dc5f5aa73f9df7a8c8d7ce03d296a3ccb5eec5fc28717f2de4874
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-03,5,5,4024152101,e2a7555f7cabd6e31aef45cb8cda4999,b969b01c158ebfecd0ac568aee526cb1ff8811fcdc77b4d682dab89146ad9891
7698,SGR,MANAGER,7839,1981-01-05,2850,null,30,2022-01-04,6,6,1255715586,c570c225d1fb8a72ad79995dd17a77bc,83f9d8d707524a4f56447204e344a997fff07b23b11f206eb5d02eac91385251
7782,RAVI,MANAGER,7839,1981-09-06,2450,null,10,2022-01-02,7,7,3000238442,ac5c482277858d6fe45065d0a3f92b0c,d7b6fab9aa91943de418ecbacefa4b276e82fbbb07bae1f7296775cc59a6f323
7788,SCOTT,ANALYST,7566,1987-04-19,3000,null,20,2022-01-02,8,8,1375856756,866c7ee013c58f01fa153a8d32c9ed57,16740bf13991fe083fbe5820cc8da08a5d88e5a48f44a3cfcc283c27b2797ba7
7839,KING,PRESIDENT,null,1981-11-01,5000,null,10,2022-01-02,9,9,3450750484,ca91c5464e73d3066825362c3093a45f,cc87d27285025584dc9cf888e8b1f415eca13f64bc04b9acbdc04259e9f27864
7844,TURNER,SALESMAN,7698,1981-08-09,1500,0,30,2022-01-02,10,10,4234062958,b356e7aed7ee82589e54a466e0dca157,b513a7ff5978b95883e4e6f83d4d991de3b8a22827d2f5bc58986e3cee03c6b1
